<a href="https://colab.research.google.com/github/agroimpacts/VegMapper/blob/master/calval/process_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a single training/validation/test set from multiple Collect Earth projects



### Table of Contents

* [Overview](#overview)
* [Using this notebook](#using-this-notebook)
* [Sample preparation](#sample-prep)
    * [Read-in, reshape, and recode](#reshape-recode)
    * [Simplify the classes](#simplify)
    * [Calculate sample agreement](#agreement)
* [Split the dataset](#split)
    * [Combine and convert to spatial](#combine)

## Overview <a class="anchor" id="overview"></a>
This notebook demonstrates how several Collect Earth Online projects can be:

1. Provide functionality to check the structure and validity of user input for modeling; Yet, users are responsible for providing data in good format.
2. Re-code the class values and rename the column names.
2. Merged into a single dataset that provide a single label for each sample point and an estimate of label uncertainty;
3. Split into training, validation, and test (or map reference) samples.

The example data that can be used for this notebook, if you don't bring your own, are the results of three Collect Earth Online projects that were captured over the Department of Ucayali, Peru. They are in the VegMapper repo under the calval/data folder. Each project csv represents the efforts of an individual (or group of individuals working in the same project) to label 1350 points, classifying each into 1 of 4 classes: not oil palm; young oil palm; mature oil palm;  unsure. The datasets preserve all the information from these projects, although user email addresses were anonymized.


### Using this notebook <a class="anchor" id="using-this-notebook"></a>

This notebook is primarily designed to run on Google Colab, and is designed to be fairly automated with no need for changing parameters or code. As such the code is hidden by default. A number of prompts for input are provided. Occassionally the prompts do not work correctly, and the dialog is not displayed. In such cases, you can click the run button to stop the execution and re-run, which should be successful or one or two tries. 

## Sample preparation <a class="anchor" id="sample-preparation"></a>
Load packages, setup configuations, define a helper function...

In [4]:
#@title (RUN) Setup code

import os
import sys

try:
    from google.colab import drive
    root = '/content/gdrive'
    drive.mount(root)
    repo_path = f"{root}/MyDrive/repos"
    clone_path = 'https://github.com/NaiaraSPinto/VegMapper.git'
    if not os.path.exists(repo_path):
        print(f"Making {repo_path}")
        os.makedirs(repo_path, exist_ok=True)

    if not os.path.exists(f"{repo_path}/VegMapper"):
        !git -C "{repo_path}" clone "{clone_path}"
    else:
        !git -C "{repo_path}/VegMapper" pull

    os.chdir(f"{repo_path}/VegMapper")

except ImportError:
    # Start from current directory and move up until 'VegMapper' is found
    curr = os.path.abspath(os.getcwd())
    while True:
        if os.path.basename(curr) == "VegMapper":
            root = curr
            repo_path = root
            os.chdir(root)
            break
        parent = os.path.dirname(curr)
        if parent == curr:
            raise FileNotFoundError("VegMapper directory not found in path.")
        curr = parent

# from datetime import datetime as dt
import pandas as pd
from sklearn.model_selection import train_test_split
# Import sample_utils function
from vegmapper.calval.label_utils import *
from functools import partial
import folium

To load the CSV files, you only need to open the directory on the left panel of your Colab notebook. Then, navigate to the directory where you have the files, click on the three dots menue to the right of the file names, and select 'Copy Path.' Finally, paste the path in the box below.

In [5]:
# @title (RUN) Load CEO Project CSVs
fs = project_file_selector()

**Note**: it is important to make sure that the CEO project files come from the same project, such each plot_id represents the same location. It is theoretically possible that 2 or more projects could have the same numbers of plots and plot_id numbers, but each plot_id represents a different location, in which case the results here will not be valid.

In [6]:
# @title (RUN) Check that projects have matching observations/columns
match_ceo_projects(fs)

Only one file provided. No comparison needed.


In [7]:
# @title (RUN) Select the columns containing the class variable
new_col_names, rename_dict = select_columns(fs)


Column Names and Indices:
0: plotid
1: center_lon
2: center_lat
3: shape
4: size_m
5: sample_points
6: email
7: flagged
8: flagged_reason
9: collection_time
10: analysis_duration
11: common_securewatch_date
12: total_securewatch_dates
13: pl_strata_cat
14: pl_cat_0
15: Oil Palm?:presence
16: Oil Palm?:absence
17: Oil Palm?:not sure
18: What is the stage? :Young Oil Palm
19: What is the stage? :Mature Oil Palm 
20: What is the stage? :not sure
{'plotid': 'Point_ID', 'pl_strata_cat': 'Clust', 'center_lat': 'Lat', 'center_lon': 'Lon', 'Oil Palm?:presence': 'Presence', 'Oil Palm?:absence': 'Absence', 'Oil Palm?:not sure': 'Unsure'}
['Presence', 'Absence', 'Unsure']


In [8]:
# @title Provide a numerical code presence (e.g. 1), absence (e.g. 0) and unsure (e.g. 2, if present)
recode_dict = {}

# Iterate through new_col_names and get user input for values
for column_name in new_col_names:
    arbitrary_number = int(
        input(f"Enter a number for the '{column_name}' category: ")
    )
    recode_dict[column_name] = arbitrary_number

print("Updated recode_dict:")
print(recode_dict)

Updated recode_dict:
{'Presence': 1, 'Absence': 0, 'Unsure': 2}


### Read-in, reshape, and recode classes <a class="anchor" id="reshape-recode"></a>
The first step is to combine the three datasets into a single dataset, with the columns from each of the three CEO projects, and to recode the four classes into a single column with values 0 (absence),  1 (presence), and 2 (unsure, if this category exists). After this step, there will be one column per CEO project. Each column contains the recoded classes, and the column is named for the CEO project.
When a category value for a CEO project is -99, it means the user didn't choose a category for that specific sample.

In [9]:
# @title (RUN) Process and combine csvs
label_name = "labeler"

if len(fs) == 1:
    dat = process_csv(fs[0], rename_dict, recode_dict, new_col_names,
                      key_col=["Point_ID", "Clust", "email"])
    labelers = list(dat["email"].unique())
    dats = [dat.query("email==@labeler").drop(columns="email", axis=0)
            for labeler in labelers]
else:
        # Define a partial function with fixed arguments
    process_csv_partial = partial(process_csv, rename_dict=rename_dict,
                                  recode_dict=recode_dict,
                                  new_col_names=new_col_names)

    # Process ceo-survey-users one by one
    dats = list(map(process_csv_partial, fs))

# Combine three datasets into one
num_labelers = len(dats)
columns_to_int = [f"ceo-survey-user{d}" for d in range(1, num_labelers+1)]
combined = combine_labelers(dats, by=["Point_ID", "Clust"],
                            label_name=label_name)
combined_pl = combined.drop(columns=['Clust'])
# columns_to_int = ['ceo-survey-user1', 'ceo-survey-user2', 'ceo-survey-user3']
combined_pl[columns_to_int] = combined_pl[columns_to_int]\
    .apply(lambda x: x.fillna(-99).astype(int))
combined_pl.head()

processing: /Users/LEstes/Dropbox/projects/VegMapper/calval/data/ceo-101_Oil_Palm_Survey_Brazil-plot-data-2025-07-11.csv


,Point_ID,Lat,Lon,ceo-survey-user1,ceo-survey-user2,ceo-survey-user3,ceo-survey-user4
0,0,-2.781842,-48.844411,1,1,1,-99
1,1,-2.040047,-47.250081,0,0,0,-99
2,2,-3.116649,-48.264190,0,0,0,-99
3,3,-1.512632,-48.377916,0,0,0,-99
4,4,-1.149183,-47.668337,1,1,1,-99


### Calculate sample agreement <a class="anchor" id="agreement"></a>
The next step is to calculate some agreement metrics across the various projects. The primary approach is to calculate the proportion of projects that selected the most common class (e.g. if there were three projects, and in two of the projects the observers labelled a particular sample as belonging to class 1 and the third labelled it as class 0, the sample is classified as 1, the modal class). If there are three projects, values can be either 0.333, 0.667, or 1. If there are four, they can 0.25, 0.5, 0.75, or 1.

Although there may be multiple classes representing presence and absence, agreement is calculated only for the simplifed classes.

In [10]:
# @title (RUN) Agreement mode
# labels = []

# for i in range(1, num_labelers + 1):
#     file_name = os.path.splitext(os.path.basename(fs[i-1]))[0]
#     labels.append(file_name)
combined[['class', 'confidence', 'count']] = combined[columns_to_int]\
    .apply(get_mode_occurence, axis=1, result_type='expand')
# combined[['class', 'confidence']] = combined[labels]\
#     .apply(get_mode_occurence, axis=1, result_type='expand')
pd.set_option('display.max_rows', None)
#print(combined)

# # we can set the mode to -9999 if there is no agreement
# #(mode_freq = 1/num_labelers)
combined.loc[combined['confidence'] <= 1/num_labelers, 'class'] = -9999
combined = combined.drop(combined[combined['class'] == -9999].index)
combined_shp = list(combined.shape)
print(f"Combined project has {combined_shp[0]} rows, {combined_shp[1]} columns")

# columns_to_int = ['ceo-survey-user1', 'ceo-survey-user2', 'ceo-survey-user3',\
#                   'class']
columns_to_int2 = columns_to_int
columns_to_int2.append("class")
combined[columns_to_int2] = combined[columns_to_int2]\
    .apply(lambda x: x.fillna(-99).astype(int))
combined_pl2 = combined.drop(columns=['Clust'])
combined_pl2.head()

Combined project has 337 rows, 11 columns


,Point_ID,Lat,Lon,ceo-survey-user1,ceo-survey-user2,ceo-survey-user3,ceo-survey-user4,class,confidence,count
0,0,-2.781842,-48.844411,1,1,1,-99,1,1.0,3.0
1,1,-2.040047,-47.250081,0,0,0,-99,0,1.0,3.0
2,2,-3.116649,-48.264190,0,0,0,-99,0,1.0,3.0
3,3,-1.512632,-48.377916,0,0,0,-99,0,1.0,3.0
4,4,-1.149183,-47.668337,1,1,1,-99,1,1.0,3.0


We can then calculate the average agreement per sample to get a sense of the uncertainty in labels for each class, for all 4 classes

In [13]:
# @title (RUN) Calculate average agreement per sample
agreement = combined.groupby("class").mean()
agreement = agreement.rename(columns={"confidence": "mean confidence"})
print(agreement[['mean confidence']])

       mean confidence
class                 
0             0.982804
1             0.901961


## Split the dataset <a class="anchor" id="split"></a>

Here we split the dataset into three parts for model training, validation, and final assessment (the portion set aside as the test or map reference dataset), according to the proportions you specify.

The splits are confined to the usable sample, which is defined as samples not falling into the unsure class and those where there was modal agreement on the class. The resulting splits are denoted in a column called `usage`.

Values of "unusable" in the `usage` column indicate observations that were not usable because of their low agreement or classified as unsure.  They are included here for completeness, and in case they help with evaluation


In [14]:
# @title (RUN) the splitting function
while True:
    train_split = float(
        input("What proportion of the sample should be assigned to training?: ")
    )
    validation_split = float(
        input("What proportion should be assigned to validation?: ")
    )
    test_split = float(
        input("What proportion of the sample should be assigned to "\
              "test/reference?: ")
    )

    # Ensure that the splits sum to 1 or are less than 1
    split_sum = train_split + validation_split + test_split

    if abs(split_sum - 1) < 1e-9:
        break
    else:
        print(f"The splits sum to {split_sum}, but must equal 1. \n"\
              "Please try again.")

seed = 999

n_samples = len(combined)
n_train = int(n_samples * train_split)
n_val = int(n_samples * validation_split)
n_test = n_samples - n_train - n_val

train = combined.sample(n_train, random_state=seed)
remaining = combined.drop(train.index)
val = remaining.sample(n_val, random_state=seed)
ref = remaining.drop(val.index)

out = pd.concat(
    [train.assign(usage="train"), val.assign(usage="validate"),
     ref.assign(usage="map_reference/test")]
).reset_index(drop=True)
out_pl = out.drop(columns=['Clust'])

out_pl.head()

,Point_ID,Lat,Lon,ceo-survey-user1,ceo-survey-user2,ceo-survey-user3,ceo-survey-user4,class,confidence,count,usage
0,297,-0.707540,-48.910977,0,-99,0,0,0,1.0,3.0,train
1,21,-0.751734,-49.271830,0,0,0,-99,0,1.0,3.0,train
2,370,-3.080051,-48.417801,0,-99,0,0,0,1.0,3.0,train
3,298,-2.962446,-48.395972,0,-99,0,0,0,1.0,3.0,train
4,93,-2.618710,-48.810725,1,-99,1,1,1,1.0,3.0,train


### Combine and export to csv <a class="anchor" id="combine"></a>

Enter the folder you want to write followed by the filename (please add ".csv" to the end).

The ineligible portion of the sample is also added back for completeness

In [15]:
#@title (RUN) Export sample
gdrive_folder = input(f"Enter the name of the output folder: \n\n")
csv_name = input(f"Enter the name of the output csv file: \n\n")

if "gdrive" in root:
    output_dir = f"{root}/MyDrive/{gdrive_folder}"
else:
    output_dir = f"{root}/{gdrive_folder}"

os.makedirs(output_dir, exist_ok=True)

outpath = os.path.join(output_dir, csv_name)

with open(outpath, 'w') as f:
    out.to_csv(f, float_format='{:f}'.format, encoding='utf-8', index=False)

print('file exported')

file exported


And their locations on a map

In [16]:
map_merged_results(out)